In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.gui.utils import tables, Tables
from api.gui.data import fields, post_process
import utils

In [ ]:
tables = Tables()
timestamp = f"{pd.Timestamp.today():%Y%m%d_%H%M%S}"

In [ ]:
root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

### Bloomberg Connection

In [ ]:
bq = BlpQuery().start()

In [ ]:
bonds_ref = pd.read_csv(data_path / 'bonds_reference.csv')

In [ ]:
searches = [
    'at1',
    'lt2',
    'sp',
    'snp'
]

In [ ]:
tickers = []
for search in searches:
    tickers.append(bq.bsrch(f"FI:{search}_daily"))

In [ ]:
tickers = pd.concat(tickers, keys=searches, sort=False).droplevel(1).rename('security')
tickers.to_csv(data_path / 'temp' / f"searches_{timestamp}.csv" )
tickers

at1    BV572427 Corp
at1    BV479212 Corp
at1    BV630090 Corp
at1    BV740538 Corp
at1    BT306419 Corp
           ...      
snp    BV261617 Corp
snp    BW467877 Corp
snp    BU611306 Corp
snp    BW647664 Corp
snp    BW667446 Corp
Name: security, Length: 192, dtype: object

In [ ]:
new_tickers = tickers.loc[~tickers.isin(bonds_ref['security'])]
new_tickers

lt2    BW893109 Corp
snp    BW647690 Corp
snp    BW647683 Corp
snp    BV170917 Corp
snp    BV979639 Corp
snp    BV170919 Corp
snp    BV339241 Corp
snp    BV992568 Corp
snp    BV992570 Corp
snp    BW048487 Corp
snp    BV261617 Corp
snp    BW467877 Corp
snp    BW647664 Corp
Name: security, dtype: object

In [ ]:
if not new_tickers.empty:
    res = bq.bdp(
        securities=new_tickers.to_list(),
        fields=[*fields],
    )
    res.to_csv(data_path / 'temp' / f"bonds_reference_{timestamp}.csv" , index=False)
    res = post_process(res, tickers)
    res = pd.concat([bonds_ref, res])
    res.to_csv(data_path / 'bonds_reference.csv', index=False)